In [1]:
try:
    import pandas as pd
    import time as time
    import database as db
    from bs4 import BeautifulSoup
    from unidecode import unidecode
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.common.keys import Keys 
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
except Exception as e:
    print(e)

In [2]:
data = db.DataBase()

In [11]:
def getData(ano, formacao, inicio=0, limitePaginas=None, curriculo=1, quantidadeMax=10):
    op = Options()
    op.headless = True
    navegador = webdriver.Chrome(options=op)
    navegador.get("http://buscatextual.cnpq.br/buscatextual/busca.do")
    
    print("Informações de pesquisa.")

    #Tags de pesquisa
    if formacao != "doutorado":
        navegador.find_element_by_xpath('//*[@id="buscarDemais"]').send_keys(Keys.SPACE) #Buscar demais pesquisadores

        if formacao == "mestrado":
            navegador.find_element_by_xpath('//*[@id="filtro2"]').click()
            time.sleep(1)
            navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').click()
            navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').send_keys(Keys.DOWN)
            navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').send_keys(Keys.DOWN)
            navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').send_keys(Keys.DOWN)
            navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').send_keys(Keys.ENTER)
            time.sleep(1)
            navegador.find_element_by_xpath('//*[@id="preencheCategoriaNivelBolsa"]')

    navegador.find_element_by_xpath('//*[@id="buscarEstrangeiros"]').click()
            
    navegador.find_element_by_xpath('//*[@id="buscaAssunto"]').send_keys(Keys.SPACE) #Buscar por assunto

    #Caixa de texto
    buscar = navegador.find_element_by_xpath('//*[@id="textoBusca"]')
    buscar.send_keys(ano)
    buscar.send_keys(Keys.ENTER)

    time.sleep(2)

    passadas = []

    janelaAtual = navegador.current_window_handle
    passadas.append(navegador.window_handles)

    #Variaveis de controle da paginação
    pagina = 1
    pag = 2
    troca = True
    delay_max = 180
    
    if limitePaginas==None:
        WebDriverWait(navegador, delay_max).until(EC.presence_of_element_located((By.XPATH, '/html/body/form/div/div[4]/div/div/div/div[3]/div/div[1]/b[1]')))
        limitePaginas = int(navegador.find_element_by_xpath('/html/body/form/div/div[4]/div/div/div/div[3]/div/div[1]/b[1]').get_attribute("innerHTML").splitlines()[0])
        limitePaginas = int(paginasTotais/10 if paginasTotais%10==0 else (paginasTotais/10)+1)
    
    if inicio!=0:
        WebDriverWait(navegador, delay_max).until(EC.presence_of_element_located((By.XPATH, f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{pag}]')))
        navegador.find_element_by_xpath(f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{pag}]').click()
        pagina = inicio
        link = navegador.current_url.split(";")
        link[0] = link[0].replace(str(10), str((pagina-1)*10))
        link = link[0]+";"+link[1]

        pag = inicio+1
        if inicio<11:
            pag = inicio+1
        elif inicio%10==0:
            pag = 13
        else:
            div = "1"+("0"*(len(str(inicio))-1))
            pag = int((inicio+1)/int(div))+2
        navegador.get(link)
        troca = False
        inicio = 0
    
    while(True):
        #Troca de pagina
        print(f"Número da pagina: {pagina}")
        
        if troca:
            try:
                if pagina!=1 and pag<=11 and pagina<=11:
                    WebDriverWait(navegador, delay_max).until(EC.presence_of_element_located((By.XPATH, f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{pag}]')))
                    navegador.find_element_by_xpath(f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{pag}]').click()
                    pag = pag+1 if pag<11 else 3

                if pagina>11 and pag>2:
                    WebDriverWait(navegador, delay_max).until(EC.presence_of_element_located((By.XPATH, f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{pag}]')))
                    navegador.find_element_by_xpath(f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{pag}]').click()
                    pag = pag+1 if pag<13 else 3
            except Exception as e:
                print(e)
                break

        #time.sleep(5)
        
        troca = True
        endFor = False
        for i in range(curriculo,11):
            try:
                #Selecionar curriculo
                WebDriverWait(navegador, delay_max).until(EC.presence_of_element_located((By.XPATH, f'//html/body/form/div/div[4]/div/div/div/div[3]/div/div[3]/ol/li[{i}]/b/a')))
                navegador.find_element_by_xpath(f'//html/body/form/div/div[4]/div/div/div/div[3]/div/div[3]/ol/li[{i}]/b/a').click()
                
                time.sleep(1)

                #Abrir curriculo
                WebDriverWait(navegador, delay_max).until(EC.presence_of_element_located((By.XPATH, '//*[@id="idbtnabrircurriculo"]')))
                navegador.find_element_by_xpath('//*[@id="idbtnabrircurriculo"]').click()
            
            except Exception as e:
                print(e)
                endFor = True
                break
                
            time.sleep(2)

            try:
                for janela in navegador.window_handles:
                    if janela not in passadas:
                        navegador.switch_to.window(janela)

            except:
                navegador.switch_to.window(janelaAtual)
                navegador.get(navegador.current_url)
                i -= 1
                continue
            
            try:
                #Pegar dados
                WebDriverWait(navegador, delay_max).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[3]/div/div/div/div[1]/h2'))) 
                site = BeautifulSoup(navegador.page_source, 'html.parser')
                artigos = []

                for artigo in site.findAll('div', attrs={'class':['artigo-completo']}):
                    if "2021" in artigo.text:
                        artigo = artigo.text[10:-4:]
                        text = ""
                        maior = 0
                        for s in artigo.split("."):
                            for ss in s.split(","):
                                if len(ss) > maior:
                                    text = ss
                                    maior = len(ss)
                        artigos.append(text)

                nome = site.findAll('h2', attrs={'class':['nome']})[0].text
            except:
                #navegador.close()
                time.sleep(1)
                #navegador.switch_to.window(janelaAtual)
                i-=1
                continue
            
            #Inserir no banco de dados
            if artigos != []:
                data.insertData(nome, formacao, artigos, ano)

            navegador.close()

            time.sleep(1)

            navegador.switch_to.window(janelaAtual)

            #Fechar curriculo
            WebDriverWait(navegador, delay_max).until(EC.presence_of_element_located((By.XPATH, '//*[@id="idbtnfechar"]')))
            navegador.find_element_by_xpath('//*[@id="idbtnfechar"]').click()
        
        if endFor:
            break
        
        data.connection.commit()
        
        #Limitador de pagínas
        if pagina == limitePaginas:
            print("Fim")
            break
        
        pagina += 1
        curriculo = 1
        
    navegador.quit()
    if pagina == limitePaginas:
        return pagina, i
    else:
        return getData(ano, formacao, inicio=pagina, curriculo=i, limitePaginas=limitePaginas)

In [21]:
try:
    print("---------- 2018 ----------")
    p18, i18 = getData("2018", "doutorado", limitePaginas=100)
except Exception as e:
    print(e)

try:
    print("---------- 2019 ----------")
    p19, i19 = getData("2019", "doutorado", limitePaginas=100, inicio=69)
except Exception as e:
    print(e)

---------- 2019 ----------
Informações de pesquisa.
Número da pagina: 69
Número da pagina: 70
Número da pagina: 71
Número da pagina: 72
Número da pagina: 73
Número da pagina: 74
Número da pagina: 75
Número da pagina: 76
Número da pagina: 77
Número da pagina: 78
Número da pagina: 79
Número da pagina: 80
Número da pagina: 81
Número da pagina: 82
Número da pagina: 83
Número da pagina: 84
Número da pagina: 85
Número da pagina: 86
Número da pagina: 87
Número da pagina: 88
Message: 

Informações de pesquisa.
Número da pagina: 88
Número da pagina: 89
Número da pagina: 90
Número da pagina: 91
Número da pagina: 92
Número da pagina: 93
Message: element click intercepted: Element <a href="javascript:abreDetalhe('K4704020T8','Eliane_Maria_Pinto_Pedrosa',293285,)">...</a> is not clickable at point (140, 572). Other element would receive the click: <div class="moldal moldalHeigh" style="height: 912px;">...</div>
  (Session info: headless chrome=94.0.4606.81)

Informações de pesquisa.
Número da pagin

In [22]:
data.selectData(2018)

,primeiro_nome,nome_completo,formacao,titulo,ano
0,Paulo,Paulo Broniera Junior,doutorado,EEG-FES-Force-MMG closed-loop control systems...,2018
1,Cláudia,Cláudia Queiroz de Vasconcelos,doutorado,Avaliação da compacidade em habitações com di...,2018
2,Cláudia,Cláudia Queiroz de Vasconcelos,doutorado,PHYSICAL AND MECHANICAL CHARACTERIZATION OF P...,2018
3,Adalberon,Adalberon Moreira de Lima Filho,doutorado,Análise dos projetos pedagógicos dos cursos d...,2018
4,Adalberon,Adalberon Moreira de Lima Filho,doutorado,¿De Olho no Velho Chico¿: uma proposta de jog...,2018
...,...,...,...,...,...
788,Andrea,Andrea Cheble de Oliveira,doutorado,Inactivation of avian influenza viruses by hy...,2018
789,Andrea,Andrea Cheble de Oliveira,doutorado,Influence of iron binding in the structural s...,2018
790,Rafael,Rafael Kunst,doutorado,Stay-at-home policy is a case of exception fa...,2018
791,Rafael,Rafael Kunst,doutorado,UFollower: A Model for Smart Cities Based on ...,2018


In [23]:
data.selectData(2019)

,primeiro_nome,nome_completo,formacao,titulo,ano
0,Galdino,Galdino Luiz Ramos Junior,doutorado,A inserção definitiva do direito civil e proc...,2019
1,Galdino,Galdino Luiz Ramos Junior,doutorado,QUEM SÃO OS DONOS DO PODER?,2019
2,Anderson,Anderson Mutter Teixeira,doutorado,The transmission mechanisms of macroprudentia...,2019
3,Anderson,Anderson Mutter Teixeira,doutorado,Prudential measures and their adverse effects...,2019
4,Maciel,Maciel Manoel de Queiroz,doutorado,Social capital and individual entrepreneurial...,2019
...,...,...,...,...,...
848,João,João Paulo Ferreira Santos,doutorado,Melt-mixed nanocomposites of SIS/MWCNT: rheol...,2019
849,João,João Paulo Ferreira Santos,doutorado,3D printability of highly ductile PETG-EMAA b...,2019
850,João,João Paulo Ferreira Santos,doutorado,PBAT/PLA/cellulose nanocrystals biocomposites...,2019
851,Carlos,Carlos Eduardo de Araujo Placido,doutorado,A presença constante da Segunda Guerra Sudane...,2019
